<a href="https://colab.research.google.com/github/Irina-64/Deep-Learning-Algorithms/blob/main/%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D0%BE%D0%B5_%D0%94%D0%97_FlowerClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ. Классификация цветов

### Продолжим работать с [Датасетом](https://www.kaggle.com/alxmamaev/flowers-recognition ) для классификации цветов (тюльпан, ромашка, подсолнух, роза, одуванчик).

Подключаем google drive чтобы хранить на нем папку с изображениями

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Скачиваем архив по ссылке, распаковываем и сохраняем в google drive в папку flowers

In [ ]:
import os
if not os.path.exists('flowers.zip'):
  !wget https://www.dropbox.com/s/qmfzqc056oiwy8c/flowers.zip
if not os.path.exists('drive/MyDrive/flowers'):
  !unzip flowers.zip -d drive/MyDrive/

# Подготовка датасета и функции для обучения

Загружаем библиотеки. Фиксируем random.seed для воспроизводимости

In [ ]:
import numpy as np # linear algebra
import os
import torch
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import random
from tqdm import tqdm

random.seed(0)
torch.manual_seed(0)

Выбираем на чем будем делать вычисления - CPU или GPU (cuda)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
prepare_imgs = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)), #приводим картинки к одному размеру
        torchvision.transforms.ToTensor(), # упаковывем их в тензор
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] # нормализуем картинки по каналам
        ),
    ]
)
# задаем датасет. Лейблы - имена папок:
dataset = ImageFolder('drive/MyDrive/flowers', transform=prepare_imgs)

In [ ]:
dataset.imgs[2]

('drive/MyDrive/flowers/daisy/10172379554_b296050f82_n.jpg', 0)

In [ ]:
class ValueMeter(object):
  """
  Вспомогательный класс, чтобы отслеживать loss и метрику
  """
  def __init__(self):
      self.sum = 0
      self.total = 0

  def add(self, value, n):
      self.sum += value*n
      self.total += n

  def value(self):
      return self.sum/self.total

def log(mode, epoch, loss_meter, accuracy_meter, best_perf=None):
  """
  Вспомогательная функция, чтобы
  """
  print(
      f"[{mode}] Epoch: {epoch:0.2f}. "
      f"Loss: {loss_meter.value():.2f}. "
      f"Accuracy: {100*accuracy_meter.value():.2f}% ", end="\n")

  if best_perf:
      print(f"[best: {best_perf:0.2f}]%", end="")


### Задаем параметры и функцию для обучения. Разбиваем датасет на train/validation

In [ ]:
batch_size = 32 # размер батча
lr = 0.001 # learning rate

Разбиваем датасет на train и validation

Задаем dataloader'ы - объекты для итеративной загрузки данных и лейблов для обучения и валидации

In [ ]:
train_set, val_set = torch.utils.data.random_split(dataset, [len(dataset)-1000, 1000])
print('Размер обучающего и валидационного датасета: ', len(train_set), len(val_set))
loaders = {'training': DataLoader(train_set, batch_size, pin_memory=True,num_workers=2, shuffle=True),
           'validation':DataLoader(val_set, batch_size, pin_memory=True,num_workers=2, shuffle=False)}

Размер обучающего и валидационного датасета:  3317 1000


Функция для подсчета Accuracy

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

Функция для обучения и валидации модели

In [ ]:
def trainval(model, loaders, optimizer, epochs=10):
    """
    model: модель, которую собираемся обучать
    loaders: dict с dataloader'ами для обучения и валидации
    optimizer: оптимизатор
    epochs: число обучающих эпох (сколько раз пройдемся по всему датасету)
    """
    loss_meter = {'training': ValueMeter(), 'validation': ValueMeter()}
    accuracy_meter = {'training': ValueMeter(), 'validation': ValueMeter()}

    loss_track = {'training': [], 'validation': []}
    accuracy_track = {'training': [], 'validation': []}

    for epoch in range(epochs): # итерации по эпохам
        for mode in ['training', 'validation']: # обучение - валидация
            # считаем градиаент только при обучении:
            with torch.set_grad_enabled(mode == 'training'):
                # в зависимоти от фазы переводим модель в нужный ружим:
                model.train() if mode == 'training' else model.eval()
                for imgs, labels in tqdm(loaders[mode]):
                    imgs = imgs.to(device) # отправляем тензор на GPU
                    labels = labels.to(device)
                    bs = labels.shape[0]  # размер батча (отличается для последнего батча в лоадере)

                    preds = model(imgs) # forward pass - прогоняем тензор с картинками через модель
                    loss = F.cross_entropy(preds, labels) # считаем функцию потерь
                    acc = accuracy(preds, labels) # считаем метрику

                    # храним loss и accuracy для батча
                    loss_meter[mode].add(loss.item(), bs)
                    accuracy_meter[mode].add(acc, bs)

                    # если мы в фазе обучения
                    if mode == 'training':
                        optimizer.zero_grad() # обнуляем прошлый градиент
                        loss.backward() # делаем backward pass (считаем градиент)
                        optimizer.step() # обновляем веса
            # в конце фазы выводим значения loss и accuracy
            log(mode, epoch, loss_meter[mode], accuracy_meter[mode])

            # сохраняем результаты по всем эпохам
            loss_track[mode].append(loss_meter[mode].value())
            accuracy_track[mode].append(accuracy_meter[mode].value())
    return loss_track, accuracy_track

In [ ]:
def set_parameter_requires_grad(model):
  """
  Функция для заморозки весов модели
  """
  for param in model.parameters():
    param.requires_grad = False

# Домашнее задание

В домашнем задании попробуем обучить еще одну сверточную архитектуру.

1) Разберите более детально код выше (или код с практики, они идентичны). Запустите весь код в этом ноутбуке.


2) Посмотрите как задаются [модели сверточных сетей в PyTorch](https://pytorch.org/vision/stable/models.html), выберите одну модель из списка:

*   alexnet
*   vgg16
*   mobilenet_v2
*   mobilenet_v3_small
*   densenet

3) Дообучите выбранную модель для задачи классификации цветов, по аналогии с тем, как мы дообучали resnet18 на занятии. Чтобы это сделать замените ____ в ячейках ниже на код. Где-то надо вставить нужную переменную, где-то прописать константу. Главное не пугайтесь! Аналогичный код можно найти в ноутбуке с практикой ;)

4) Если вам интересно, прочитайте дополнительно про эти архитектуры. Можно почитать [здесь (на английском)](https://medium.com/@fransiska26/the-differences-between-inception-resnet-and-mobilenet-e97736a709b0) или [здесь (на русском)](https://habr.com/ru/company/nix/blog/430524/)

**Совет 1:** загрузите папку с картинками на гугл диск, чтобы не загружать ее каждый раз заново при перезапуске колаба. Структура файлов (можно посмотреть в меню слева) должна быть такой: drive/MyDrive/flowers

**Совет 2:** обязательно подключите аппаратный ускоритель (GPU) к среде выполнения, чтобы вычисления были. В меню сверху: Среда выполнения -> Сменить среду выполнения



In [ ]:
# Выберите модель из списка доступных в PyTorch моделей
# Не забудьте указать, что она модель должна быть предобучена!
model = torchvision.models._____
model

In [ ]:
set_parameter_requires_grad(_____) # передайте модель в функцию для "заморозки" градиента

In [ ]:
model._____ = _____# Меняем последний слой модели

In [ ]:
# Проверим все ли сработало правильно, выведем веса, которые будут обучаться
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

In [ ]:
model.to(_____) # Отправляем модель на GPU
optimizer = torch.optim.Adam(params = model.parameters()) # алгоритм оптимизации
loss_track, accuracy_track = trainval(model, loaders, optimizer, epochs=_____) # запускаем обучение, задаем количество эпох